In [ ]:
import cv2
import numpy as np
from PIL import Image
from datasets.load import load_dataset
from pycocotools import mask as maskUtils

In [ ]:
dataset = load_dataset(
    'coco2017', 'keypoints', trust_remote_code=True
)

In [ ]:
def transforms(examples):
    for objects in examples['objects']:
        if objects:
            if objects['instance_rles']:
                instance_rles = objects['instance_rles']
                objects['masks'] = [
                    maskUtils.decode({'size': size, 'counts': counts})
                    for size, counts in zip(
                        instance_rles['size'], instance_rles['counts']
                    )
                ]
    return examples

dataset.set_transform(transforms)

In [ ]:
names = dataset['train'].features['objects']['categories'].feature.names

In [ ]:
colors = np.ones((80, 3), np.uint8) * 255
colors[:, 0] = np.linspace(0, 255, 80)
colors = cv2.cvtColor(colors[None], cv2.COLOR_HSV2RGB)[0]

In [ ]:
def visualize(example, names, colors):
    image = np.array(example['image'])
    if example['objects']:
        objects = example['objects']
        bboxes = np.array(objects['bboxes']).round().astype(int)
        bboxes[:, 2:] += bboxes[:, :2]
        masks = objects['masks']
        n = len(objects['bboxes'])
        for i in range(n):
            c = objects['categories'][i]
            color, name = colors[c], names[c]
            cv2.rectangle(image, bboxes[i, :2], bboxes[i, 2:], color.tolist(), 2)
            cv2.putText(
                image, name, bboxes[i, :2], cv2.FONT_HERSHEY_SIMPLEX,
                1, color.tolist(), 2, cv2.LINE_AA, False
            )
            image[masks[i] == 1] = image[masks[i] == 1] // 2 + color // 2
    return image


In [ ]:
example = dataset['train'][500]
Image.fromarray(visualize(example, names, colors))